# Install library

In [ ]:
!pip install streamlit
!npm install localtunnel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 92.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.5 MB/s eta 0:00:00
npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 4.038s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit f

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Main

## Def

In [ ]:
# %%writefile files_def.py
import tensorflow as tf
import numpy as np
import random
import cv2
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import f1_score, jaccard_score, precision_score, recall_score
from tensorflow.keras.metrics import IoU
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sn
from matplotlib import pyplot
import pandas as pd

#------------- Classification----------------
def load_model(name_md):
    if name_md == "VGG16":
        model=tf.keras.models.load_model('/content/drive/MyDrive/Deep_learning/image_classification/brain_tumor/files/VGG16_classification.h5')
        return model
    if name_md == "ResNet50":
        model=tf.keras.models.load_model('/content/drive/MyDrive/Deep_learning/image_classification/brain_tumor/files/ResNet50_classification.h5')
        return model

#-------------- Segmentation----------------
def import_and_predict(image_data, model):
        size = (224,224)
        image = ImageOps.fit(image_data, size, Image.ANTIALIAS)
        image = np.asarray(image)
        img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        img = img / 255
        img = np.array([img])
        predictions = model.predict(img)[0][0]
        return predictions

def load_model_segm(name):
        if name == "Unet":
            with CustomObjectScope({"dice_coef": dice_coef, "dice_loss": dice_loss}):
                model = tf.keras.models.load_model("/content/drive/MyDrive/Deep_learning/Brain_Tumor_Segmentation_unet/1.Build_unet/files/files_2/Metrics_model_5000_unet.h5")
                return model
        else:
            with CustomObjectScope({"dice_coef": dice_coef, "dice_loss": dice_loss}):
                model = tf.keras.models.load_model("/content/drive/MyDrive/Deep_learning/Brain_Tumor_Segmentation_unet/SegNet/files/2_Metrics_model_5000_segnet.h5")
                return model

def plot_image_predicted_mask_Seg(original_img, mask, predicted_mask):

    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(10, 8))
    axes[0].imshow(original_img)
    axes[0].get_xaxis().set_visible(False)
    axes[0].get_yaxis().set_visible(False)
    axes[0].set_title('Ảnh')
    axes[1].imshow(mask)
    axes[1].get_xaxis().set_visible(False)
    axes[1].get_yaxis().set_visible(False)
    axes[1].set_title('Mask')
    axes[2].imshow(predicted_mask, cmap='gray')
    axes[2].get_xaxis().set_visible(False)
    axes[2].get_yaxis().set_visible(False)
    axes[2].set_title('Ảnh dự đoán')
    fig.tight_layout()
    filename = 'Mask_segmentation' + '.png'
    plt.savefig(filename)

    return  filename

def predictionSeg(array_img, mask, model):

        # """ Reading the image """
        image = cv2.imread(array_img, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (256, 256))
        x = image/255.0
        x = np.expand_dims(x, axis=0)

        mask = cv2.imread(mask, cv2.IMREAD_COLOR)
        mask = cv2.resize(mask, (256, 256))

        # Prediction
        y_pred = model.predict(x, verbose=0)[0]
        y_pred = np.squeeze(y_pred, axis=-1)
        y_pred = y_pred >= 0.5
        y_pred = y_pred.astype(np.int32)
        return image, mask, y_pred

def predictionNet(array_img, model):

        # """ Reading the image """
        image = cv2.imread(array_img, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (256, 256))
        x = image/255.0
        x = np.expand_dims(x, axis=0)

        # Prediction
        y_pred = model.predict(x, verbose=0)[0]
        y_pred = np.squeeze(y_pred, axis=-1)
        y_pred = y_pred >= 0.5
        y_pred = y_pred.astype(np.int32)
        return image, y_pred

def plot_image_predicted_mask_Net(original_img, predicted_mask):

                fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(7, 5))
                axes[0].imshow(original_img)
                axes[0].get_xaxis().set_visible(False)
                axes[0].get_yaxis().set_visible(False)
                axes[0].set_title('Ảnh')
                axes[1].imshow(predicted_mask, cmap='gray')
                axes[1].get_xaxis().set_visible(False)
                axes[1].get_yaxis().set_visible(False)
                axes[1].set_title('Ảnh dự đoán')
                fig.tight_layout()
                filename = 'Mask_segmentation' + '.png'
                plt.savefig(filename)

                return  filename

smooth = 1e-15
def dice_coef(y_true, y_pred):
            y_true = tf.keras.layers.Flatten()(y_true)
            y_pred = tf.keras.layers.Flatten()(y_pred)
            intersection = tf.reduce_sum(y_true * y_pred)
            return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

def metrics(path_mask, y_pred):

    # Flatten the array
    """ Reading the mask """
    mask = cv2.imread(path_mask, cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (256, 256))
    mask = mask/255.0
    mask = (mask > 0.5).astype(np.int32).flatten()
    y_pred = y_pred.flatten()

    """ Calculating the metrics values """
    f1_value = f1_score(mask, y_pred, labels=[0, 1], average="binary")
    recall_value = recall_score(mask, y_pred, labels=[0, 1], average="binary", zero_division=0)
    precision_value = precision_score(mask, y_pred, labels=[0, 1], average="binary", zero_division=0)
    m = IoU(num_classes=2, target_class_ids=[1])
    m.update_state(mask, y_pred)
    iou = m.result().numpy()

    confusion = confusion_matrix(mask, y_pred)
    df_cm = pd.DataFrame(confusion, index = [0,1],columns = [0,1])
    pyplot.figure(figsize = (10,7))
    sn.heatmap(df_cm, annot=True, fmt="g" )
    plt.savefig('confusion.png')

    return f1_value, recall_value, precision_value, iou



Writing files_def.py


## File Streamlit

In [ ]:
%%writefile Ung_Dung_Luan_Van.py
import streamlit as st
import tensorflow as tf
import numpy as np
import random
import cv2
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
from tensorflow.keras.utils import CustomObjectScope
from files_def import *
import pandas as pd

st.markdown("<h1 style='text-align: center; color: green;'>Ứng dụng Luận Văn: Đề Tài Tìm Hiểu Một Số Kỹ Thuật Về Phân Đoạn Ảnh Ngữ Nghĩa</h1>", unsafe_allow_html=True)

st.sidebar.title('🤗Dạ Lựa Chọn Chế Độ Ạ🤗')
activity = ['Classification','Segmentation',]
choice = st.sidebar.selectbox("Chế độ",activity)

if choice == 'Classification':
    st.subheader("Classification não bộ")
    option = st.selectbox('Mời bạn chọn loại não cần xem xét?',("Đã bị U não", "Não khoẻ mạnh"))
    st.write("")
    file = st.file_uploader("Mời bạn tải ảnh tại đây!")

    st.set_option('deprecation.showfileUploaderEncoding', False)

    if file is None:
        st.text("Vui lòng! tải ảnh")
    else:
        model_select = st.selectbox('Chọn kiến trúc mô hình bạn muốn sử dụng: ', ("VGG16", "ResNet50"))
        image = Image.open(file)
        st.image(image, use_column_width=True)
        model = load_model(model_select)
        predictions = import_and_predict(image, model)
        result = round(predictions)

        if result == 0 :
            text = 'Đã bị U não'
        else :
            text = 'Não khoẻ mạnh'

        if option == text:
            st.success(text)

        else:
            stri = option + ", sai !"
            st.error(stri)

if choice == 'Segmentation':
    st.subheader("Segmentation não bộ")

    mode = st.selectbox('Chọn chế độ kiểm Tra: ', ("Loại kiểm tra ", "Dử liệu Segmentation", "Trên Mạng"))
    if mode == "Dử liệu Segmentation":
        # load Image, Mask, Model
        image = st.file_uploader("Mời bạn tải ảnh tại đây!")

        mask = st.file_uploader("Mời bạn tải mask tại đây!")

        model_select = st.selectbox('Chọn kiến trúc mô hình bạn muốn sử dụng: ', ("Loại Kiến trúc ", "Unet", "SegNet"))

        if model_select == "Unet":
            # Read model
            model = load_model_segm(model_select)

            if image is not None:

                # Read Image
                image_input = Image.open(image)
                nimg = np.array(image_input)
                image = cv2.resize(nimg, (256, 256), interpolation=cv2.INTER_AREA)
                cv2.imwrite("anh_unet_input.png", image)

                # Read Mask
                mask_input = Image.open(mask)
                Mimg = np.array(mask_input)
                mask_r = cv2.resize(Mimg, (256, 256), interpolation=cv2.INTER_AREA)
                cv2.imwrite("mask_unet_input.png", mask_r)

                # Predict And Show Resualt Out Window
                image, mask_O, mask_pred = predictionSeg("anh_unet_input.png", "mask_unet_input.png",  model)

                filename = plot_image_predicted_mask_Seg(image, mask_O, mask_pred)

                f1_value, recall_value, precision_value, iou =  metrics("mask_unet_input.png", mask_pred)

                # st.subheader("Ma trận Confusion")
                # st.image("/content/confusion.png")

                # st.subheader("Các chỉ số đánh giá:")
                # st.write(pd.DataFrame({
                #     'Precision': [precision_value],
                #     'Độ nhạy (Recall)': [recall_value],
                #     'F1-Score': [f1_value],
                #     'IoU (Intersection-Over-Union)': [iou]
                # }))

                st.subheader("Kết quả dự đoán")
                st.image(filename)

                with open(filename, "rb") as file:
                    btn = st.download_button(
                            label = "Tải hình đã segmentation",
                            data = file,
                            file_name = filename,
                            mime = "/content/"
                        )
        if model_select == "SegNet":
            # Load model
            model = load_model_segm(model_select)

            if image is not None:

                # Read Image
                image_input = Image.open(image)
                nimg = np.array(image_input)
                image = cv2.resize(nimg, (256, 256), interpolation=cv2.INTER_AREA)
                cv2.imwrite("anh_unet_input.png", image)

                # Read Mask
                mask_input = Image.open(mask)
                Mimg = np.array(mask_input)
                mask = cv2.resize(Mimg, (256, 256), interpolation=cv2.INTER_AREA)
                cv2.imwrite("mask_unet_input.png", mask)

                # Predict And Show Resualt Out Window
                image, mask_p, mask_pred = predictionSeg("anh_unet_input.png", "mask_unet_input.png", model)
                filename = plot_image_predicted_mask_Seg(image, mask_p, mask_pred, )

                f1_value, recall_value, precision_value, iou =  metrics("mask_unet_input.png", mask_pred)

                # st.subheader("Ma trận Confusion")
                # st.image("/content/confusion.png")

                # st.subheader("Các chỉ số đánh giá")
                # st.write(pd.DataFrame({
                #     # 'Precision': [precision_value],
                #     # 'Độ nhạy (Recall)': [recall_value],
                #     # 'F1-Score': [f1_value],
                #     'IoU (Intersection-Over-Union)': [iou]
                # }))

                st.subheader("Kết quả dự đoán")
                st.image(filename)

                with open(filename, "rb") as file:
                    btn = st.download_button(
                            label = "Tải hình đã segmentation",
                            data = file,
                            file_name = filename,
                            mime = "/content/"
                        )
    if mode == "Trên Mạng":
        model_select = st.selectbox('Chọn kiến trúc mô hình bạn muốn sử dụng: ', ("Loại Kiến trúc ", "Unet", "SegNet"))
        if model_select == "Unet":
            # Tải model
            model = load_model_segm(model_select)

            # Tải Hình ảnh
            file = st.file_uploader("Mời bạn tải ảnh tại đây!")
            if file is not None:

                image_input = Image.open(file)
                nimg = np.array(image_input)
                image = cv2.resize(nimg, (256, 256), interpolation=cv2.INTER_AREA)
                cv2.imwrite("anh_unet_input.png", image)

                # Reading the image
                image, image_pred = predictionNet("anh_unet_input.png", model)

                filename = plot_image_predicted_mask_Net(image, image_pred)
                st.subheader("Kết quả:")
                st.image(filename)

                # Downw Load image predicted
                with open(filename, "rb") as file:
                    btn = st.download_button(
                            label = "Tải hình đã segmentation",
                            data = file,
                            file_name = filename,
                            mime = "/content/"
                        )
        if model_select == "SegNet":
            # Load model
            model = load_model_segm(model_select)

            # Load image
            file = st.file_uploader("Mời bạn tải ảnh tại đây!")
            if file is not None:

                image_input = Image.open(file)

                nimg = np.array(image_input)
                image = cv2.resize(nimg, (256, 256), interpolation=cv2.INTER_AREA)
                cv2.imwrite("anh_unet_input.png", image)
                image, image_pred = predictionNet("anh_unet_input.png", model)

                filename = plot_image_predicted_mask_Net(image,image_pred)
                st.subheader("Kết quả:")
                st.image(filename)

                with open(filename, "rb") as file:
                    btn = st.download_button(
                            label = "Tải hình đã segmentation",
                            data = file,
                            file_name = filename,
                            mime = "/content/"
                        )


Writing Ung_Dung_Luan_Van.py


# Run

In [ ]:
# Get ID
!wget -q -O - ipv4.icanhazip.com

104.196.28.31


In [ ]:
# booting server
!streamlit run /content/Ung_Dung_Luan_Van.py &>/content/logs.txt &
!npx localtunnel --port 8501

npx: installed 22 in 3.928s
your url is: https://ten-birds-suffer.loca.lt
